In [182]:
# Import libraries
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from stargazer.stargazer import Stargazer

# Redirecting to the directory
# os.chdir(os.path.join('.\ECON4999X'))
CLEANED_DATA_DIR = os.path.normpath(r'.\Cleaned_Data')
DA_DATA_DIR = os.path.normpath(r'.\da_working_files')
# print(os.listdir(CLEANED_DATA_DIR))

import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt
import seaborn as sns

### Final cleaning the dataframe

In [201]:
capitaliq_output_file_name = 'capitaliq_output.xlsx'
df_capital_iq = pd.read_excel(os.path.join(CLEANED_DATA_DIR, capitaliq_output_file_name), sheet_name=None)

In [55]:
info_file_name = 'df_info_master.csv'
df_info = pd.read_csv(os.path.join(CLEANED_DATA_DIR, info_file_name))
panel_file_name = 'df_panel_master.csv'
df_panel = pd.read_csv(os.path.join(DA_DATA_DIR, panel_file_name))

In [202]:
df_capital_iq.keys()

dict_keys(['Total asset', 'Net income', 'Fixed asset', 'fx_asset_tot_asset', 'lt debt', 'total debt', '_CIQHiddenCacheSheet', 'mkt cap', 'fx_asset_org', 'lt_debt_org', 'mkt_cap_org', 'net_income_org', 'tot_asset_org'])

In [203]:
for sheet_name in df_capital_iq.keys():
    if 'org' in sheet_name:
        print(sheet_name)
        
        df_capital_iq[sheet_name]['year'] = df_capital_iq[sheet_name]['Unnamed: 0'].dt.year
        col_list = []
        for i in df_capital_iq[sheet_name].columns:
            if ':' in i and "Unnamed" not in i:
                entity_id = df_info.query('exchange_ticker == @i')['entity_id'].values[0]
                col_list.append(entity_id)
            else:
                col_list.append(i)
        df_capital_iq[sheet_name].columns = col_list

fx_asset_org
lt_debt_org
mkt_cap_org
net_income_org
tot_asset_org


In [204]:
for sheet_name in df_capital_iq.keys():
    if 'org' in sheet_name:
        df_capital_iq[sheet_name] = df_capital_iq[sheet_name].drop(['Unnamed: 0', 'Unnamed: 1'], axis=1).melt(id_vars=['year'])
        df_capital_iq[sheet_name] = df_capital_iq[sheet_name].rename(columns={'value': sheet_name.replace('_org', ''), 'variable': 'entity_id'})
        

In [206]:
sheet_name = 'tot_asset_org'
a = df_capital_iq[sheet_name][~(df_capital_iq[sheet_name]['year'] != df_capital_iq[sheet_name]['year'])]
a['year'] = a['year'].astype('int')
a['entity_id'] = a['entity_id'].astype('int')

df_panel = df_panel.merge(a, on=['entity_id', 'year'], how='left')

C:\Users\Edrick\AppData\Local\Temp\ipykernel_5720\2252306903.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['year'] = a['year'].astype('int')
C:\Users\Edrick\AppData\Local\Temp\ipykernel_5720\2252306903.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['entity_id'] = a['entity_id'].astype('int')


In [143]:
for sheet_name in df_capital_iq.keys():
    if 'org' in sheet_name:
        a = df_capital_iq[sheet_name][~(df_capital_iq[sheet_name]['year'] != df_capital_iq[sheet_name]['year'])]
        a['year'] = a['year'].astype('int')
        a['entity_id'] = a['entity_id'].astype('int')

        df_panel = df_panel.merge(a, on=['entity_id', 'year'], how='left')

C:\Users\Edrick\AppData\Local\Temp\ipykernel_5720\766321886.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['year'] = a['year'].astype('int')
C:\Users\Edrick\AppData\Local\Temp\ipykernel_5720\766321886.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['entity_id'] = a['entity_id'].astype('int')
C:\Users\Edrick\AppData\Local\Temp\ipykernel_5720\766321886.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [209]:
df_panel[['ln_fx_asset', 'ln_lt_debt', 'ln_mkt_cap', 'ln_net_income', 'ln_tot_asset']] = np.log(df_panel[['fx_asset', 'lt_debt', 'mkt_cap', 'net_income', 'tot_asset']])

c:\Users\Edrick\PycharmProjects\sustainbale_econ_research\.venv\lib\site-packages\pandas\core\internals\blocks.py:329: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


In [157]:
df_panel

,entity_id,year,absolute_ghg_scope_1,absolute_ghg_scope_2,intensity_ghg_scope_1,intensity_ghg_scope_2,total_carbon_emission,biomass,coal,geothermal,...,is_reduced_ce,is_long_hist,fx_asset,lt_debt,mkt_cap,net_income,ln_fx_asset,ln_lt_debt,ln_mkt_cap,ln_net_income
0,4147764,2004,1.360592e+06,105613.412500,247.430227,19.206315,1.466205e+06,NaN,NaN,NaN,...,True,True,3415.050,1139.044,4101.18119,48.314,8.135947,7.037945,8.319030,3.877721
1,4147764,2005,1.289793e+06,100051.889800,225.311038,17.477839,1.389845e+06,NaN,NaN,NaN,...,True,True,3594.792,1600.031,5977.08955,68.377,8.187241,7.377778,8.695689,4.225037
2,4147764,2006,1.670383e+06,126135.647900,201.841385,15.241652,1.796518e+06,NaN,NaN,NaN,...,True,True,5410.553,9998.880,8777.53187,1050.111,8.596107,9.210228,9.079951,6.956651
3,4147764,2007,1.586784e+06,35446.030000,136.679404,3.053184,1.622230e+06,NaN,NaN,NaN,...,True,True,22033.722,15795.305,13489.77878,31.000,10.000329,9.667468,9.509688,3.433987
4,4147764,2008,1.175414e+06,167569.980000,66.636061,9.499804,1.342984e+06,NaN,NaN,NaN,...,True,True,23336.956,18338.146,5525.12418,118.471,10.057793,9.816739,8.617061,4.774668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,4539460,2016,1.161410e+07,903.453175,16951.953960,1.318682,1.161500e+07,78.00,9192.00,NaN,...,True,False,7454.462,592.145,640.88493,31.466,8.916568,6.383752,6.462850,3.448908
280,4539460,2017,1.099803e+07,1160.731868,17021.534380,1.796453,1.099919e+07,36.67,8353.33,NaN,...,True,False,7547.577,303.335,772.00968,13.875,8.928982,5.714838,6.648997,2.630089
281,4539460,2018,7.332719e+06,1057.278150,11495.301030,1.657466,7.333776e+06,40.22,6009.78,NaN,...,True,False,7586.201,59.424,381.17660,-398.109,8.934086,4.084698,5.943263,NaN
282,4539460,2019,4.226335e+06,968.032085,6098.313233,1.396804,4.227303e+06,63.82,3336.18,NaN,...,True,False,7562.173,29.773,391.34131,-517.770,8.930914,3.393602,5.969580,NaN


In [207]:
# Export the df_panel
df_panel.to_csv(os.path.join(DA_DATA_DIR, 'df_panel_master.csv'), index=False)

### Regression model 1

In [166]:
# Add carbon pricing to df_panel
cp_file_name = 'df_cp.csv'
df_cp = pd.read_csv(os.path.join(DA_DATA_DIR, cp_file_name))
df_cp['ln_cp_avg_prce'] = np.log(df_cp['cp_avg_price'])

In [215]:
df_panel['ln_abs_ghg_scope_1'] = np.log(df_panel['absolute_ghg_scope_1'])
df_panel['ln_intensity_ghg_scope_1'] = np.log(df_panel['intensity_ghg_scope_1'])
df_panel['ln_total_carbon_emission'] = np.log(df_panel['total_carbon_emission'])

In [191]:
df_panel['ln_cp_avg_prce_sq'] = np.square(df_panel['ln_cp_avg_prce'])

In [173]:
df_panel = df_panel.merge(df_cp, left_on='year', right_on='year', how='left').drop_duplicates()

In [179]:
unit_col_name='entity_id'
time_period_col_name='year'

y_var_name = 'ln_abs_ghg_scope_1'
X_var_names = ['ln_cp_avg_prce']

regr_formula = y_var_name + ' ~ ' + ' + '.join(X_var_names)

In [185]:
def stargazer_to_html(stargazer_str_html, output_file_name):
    output_dir = os.path.normpath(r'.\Outputs')
    output_path = os.path.join(output_dir, output_file_name)
    with open(output_path, 'w') as f:
        f.write(stargazer_str_html)
    return True

In [186]:
panel_fe_model = smf.ols(formula=regr_formula, data=df_panel)
panel_fe_model = panel_fe_model.fit()
print(panel_fe_model.summary())
stargazer_to_html(Stargazer([panel_fe_model]).render_html(), 'panel_fe_model_1.html')

                            OLS Regression Results                            
Dep. Variable:     ln_abs_ghg_scope_1   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.5932
Date:                Fri, 05 May 2023   Prob (F-statistic):              0.442
Time:                        12:24:11   Log-Likelihood:                -621.50
No. Observations:                 263   AIC:                             1247.
Df Residuals:                     261   BIC:                             1254.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         15.8153      0.544     29.

True

In [212]:
df_panel_long_hist = df_panel.query('is_long_hist == True')
df_panel_long_hist

,entity_id,year,absolute_ghg_scope_1,absolute_ghg_scope_2,intensity_ghg_scope_1,intensity_ghg_scope_2,total_carbon_emission,biomass,coal,geothermal,...,ln_lt_debt,ln_mkt_cap,ln_net_income,cp_avg_price,ln_cp_avg_prce,ln_abs_ghg_scope_1,ln_intensity_ghg_scope_1,ln_cp_avg_prce_sq,tot_asset,ln_tot_asset
0,4147764,2004,1360591.600,105613.4125,247.430227,19.206315,1466205.013,NaN,NaN,NaN,...,7.037945,8.319030,3.877721,NaN,NaN,14.123430,5.511129,NaN,9488.185,9.157803
1,4147764,2005,1289793.039,100051.8898,225.311038,17.477839,1389844.929,NaN,NaN,NaN,...,7.377778,8.695689,4.225037,21.436486,3.065094,14.069992,5.417482,9.394804,10911.109,9.297537
2,4147764,2006,1670382.787,126135.6479,201.841385,15.241652,1796518.435,NaN,NaN,NaN,...,9.210228,9.079951,6.956651,17.389423,2.855862,14.328563,5.307482,8.155949,21287.366,9.965869
3,4147764,2007,1586783.642,35446.0300,136.679404,3.053184,1622229.672,NaN,NaN,NaN,...,9.667468,9.509688,3.433987,1.519423,0.418331,14.277220,4.917638,0.175001,38635.424,10.561925
4,4147764,2008,1175414.112,167569.9800,66.636061,9.499804,1342984.092,NaN,NaN,NaN,...,9.816739,8.617061,4.774668,22.386731,3.108468,13.977131,4.199246,9.662576,38458.369,10.557332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,5001062,2017,1373338.003,451625.0000,392.586545,129.102885,1824963.003,2.06,777.69,NaN,...,7.448507,8.853478,3.463859,5.860000,1.768150,14.132755,5.972757,3.126353,11283.602,9.331106
272,5001062,2018,1108015.302,411000.0000,329.659777,122.281857,1519015.302,1.03,707.00,NaN,...,6.945934,9.467905,5.013145,16.076346,2.777349,13.918081,5.798061,7.713668,11704.798,9.367754
273,5001062,2019,1085552.823,392001.7670,248.958000,89.901000,1477554.590,NaN,909.13,NaN,...,6.689067,9.651390,4.643593,24.894423,3.214644,13.897600,5.517284,10.333935,11838.629,9.379123
274,5001062,2020,662079.532,406000.0180,179.205000,109.892000,1068079.550,NaN,351.01,NaN,...,6.652158,10.096872,5.035178,24.724038,3.207776,13.403141,5.188530,10.289827,11987.668,9.391634


In [245]:
unit_col_name='entity_id'
time_period_col_name='year'

y_var_name = 'ln_intensity_ghg_scope_1'
X_var_names = ['ln_cp_avg_prce', 'ln_cp_avg_prce_sq', 'ln_fx_asset', 'ln_lt_debt', 'ln_tot_asset', 'ln_net_income']

regr_formula = y_var_name + ' ~ ' + ' + '.join(X_var_names) + ' + C(year)'

panel_fe_model = smf.ols(formula=regr_formula, data=df_panel) # df_panel  df_panel_long_hist
panel_fe_model = panel_fe_model.fit()
print(panel_fe_model.summary())
# stargazer_to_html(Stargazer([panel_fe_model]).render_html(), 'panel_fe_model_6.html')

                               OLS Regression Results                               
Dep. Variable:     ln_intensity_ghg_scope_1   R-squared:                       0.379
Model:                                  OLS   Adj. R-squared:                  0.316
Method:                       Least Squares   F-statistic:                     5.950
Date:                      Fri, 05 May 2023   Prob (F-statistic):           1.42e-11
Time:                              14:23:44   Log-Likelihood:                -365.37
No. Observations:                       205   AIC:                             770.7
Df Residuals:                           185   BIC:                             837.2
Df Model:                                19                                         
Covariance Type:                  nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

In [210]:
df_panel

,entity_id,year,absolute_ghg_scope_1,absolute_ghg_scope_2,intensity_ghg_scope_1,intensity_ghg_scope_2,total_carbon_emission,biomass,coal,geothermal,...,ln_lt_debt,ln_mkt_cap,ln_net_income,cp_avg_price,ln_cp_avg_prce,ln_abs_ghg_scope_1,ln_intensity_ghg_scope_1,ln_cp_avg_prce_sq,tot_asset,ln_tot_asset
0,4147764,2004,1.360592e+06,105613.412500,247.430227,19.206315,1.466205e+06,NaN,NaN,NaN,...,7.037945,8.319030,3.877721,NaN,NaN,14.123430,5.511129,NaN,9488.185,9.157803
1,4147764,2005,1.289793e+06,100051.889800,225.311038,17.477839,1.389845e+06,NaN,NaN,NaN,...,7.377778,8.695689,4.225037,21.436486,3.065094,14.069992,5.417482,9.394804,10911.109,9.297537
2,4147764,2006,1.670383e+06,126135.647900,201.841385,15.241652,1.796518e+06,NaN,NaN,NaN,...,9.210228,9.079951,6.956651,17.389423,2.855862,14.328563,5.307482,8.155949,21287.366,9.965869
3,4147764,2007,1.586784e+06,35446.030000,136.679404,3.053184,1.622230e+06,NaN,NaN,NaN,...,9.667468,9.509688,3.433987,1.519423,0.418331,14.277220,4.917638,0.175001,38635.424,10.561925
4,4147764,2008,1.175414e+06,167569.980000,66.636061,9.499804,1.342984e+06,NaN,NaN,NaN,...,9.816739,8.617061,4.774668,22.386731,3.108468,13.977131,4.199246,9.662576,38458.369,10.557332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,4539460,2016,1.161410e+07,903.453175,16951.953960,1.318682,1.161500e+07,78.00,9192.00,NaN,...,6.383752,6.462850,3.448908,5.407170,1.687726,16.267730,9.738138,2.848418,4801.300,8.476642
280,4539460,2017,1.099803e+07,1160.731868,17021.534380,1.796453,1.099919e+07,36.67,8353.33,NaN,...,5.714838,6.648997,2.630089,5.860000,1.768150,16.213226,9.742235,3.126353,4459.380,8.402765
281,4539460,2018,7.332719e+06,1057.278150,11495.301030,1.657466,7.333776e+06,40.22,6009.78,NaN,...,4.084698,5.943263,NaN,16.076346,2.777349,15.807857,9.349694,7.713668,3871.297,8.261345
282,4539460,2019,4.226335e+06,968.032085,6098.313233,1.396804,4.227303e+06,63.82,3336.18,NaN,...,3.393602,5.969580,NaN,24.894423,3.214644,15.256846,8.715767,10.333935,3118.246,8.045026


In [208]:
df_panel.columns

Index(['entity_id', 'year', 'absolute_ghg_scope_1', 'absolute_ghg_scope_2',
       'intensity_ghg_scope_1', 'intensity_ghg_scope_2',
       'total_carbon_emission', 'biomass', 'coal', 'geothermal',
       'hydroelectric', 'landfill_gas', 'liquid_nat_gas', 'nat_gas', 'nuclear',
       'petroleum_oil', 'solar', 'wave_and_tidal', 'wind', 'green_tech_power',
       'fossil_fuel_power', 'total_power', 'green_tech_rr', 'is_reduced_ce',
       'is_long_hist', 'fx_asset', 'lt_debt', 'mkt_cap', 'net_income',
       'ln_fx_asset', 'ln_lt_debt', 'ln_mkt_cap', 'ln_net_income',
       'cp_avg_price', 'ln_cp_avg_prce', 'ln_abs_ghg_scope_1',
       'ln_intensity_ghg_scope_1', 'ln_cp_avg_prce_sq', 'tot_asset'],
      dtype='object')

In [222]:
# Adding year dummies
df_panel.join(pd.get_dummies(df_panel['year']).drop(2004, axis=1))

,entity_id,year,absolute_ghg_scope_1,absolute_ghg_scope_2,intensity_ghg_scope_1,intensity_ghg_scope_2,total_carbon_emission,biomass,coal,geothermal,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,4147764,2004,1.360592e+06,105613.412500,247.430227,19.206315,1.466205e+06,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,4147764,2005,1.289793e+06,100051.889800,225.311038,17.477839,1.389845e+06,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,4147764,2006,1.670383e+06,126135.647900,201.841385,15.241652,1.796518e+06,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,4147764,2007,1.586784e+06,35446.030000,136.679404,3.053184,1.622230e+06,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
4,4147764,2008,1.175414e+06,167569.980000,66.636061,9.499804,1.342984e+06,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,4539460,2016,1.161410e+07,903.453175,16951.953960,1.318682,1.161500e+07,78.00,9192.00,NaN,...,False,False,False,False,True,False,False,False,False,False
280,4539460,2017,1.099803e+07,1160.731868,17021.534380,1.796453,1.099919e+07,36.67,8353.33,NaN,...,False,False,False,False,False,True,False,False,False,False
281,4539460,2018,7.332719e+06,1057.278150,11495.301030,1.657466,7.333776e+06,40.22,6009.78,NaN,...,False,False,False,False,False,False,True,False,False,False
282,4539460,2019,4.226335e+06,968.032085,6098.313233,1.396804,4.227303e+06,63.82,3336.18,NaN,...,False,False,False,False,False,False,False,True,False,False


In [238]:
panel_all_file_name = 'df_ghg_ts_long_form_all_comps.csv'
df_panel_all = pd.read_csv(os.path.join(DA_DATA_DIR, panel_all_file_name))
df_panel_all

,entity_id,year,absolute_ghg_scope_1,absolute_ghg_scope_2,intensity_ghg_scope_1,intensity_ghg_scope_2,exchange,is_eu,country
0,1031123,2004,6.094485e+06,1582.414375,3167.529750,0.822439,NYSE,NaN,United States
1,1031123,2005,5.547608e+06,1891.407577,2503.925747,0.853691,NYSE,NaN,United States
2,1031123,2006,6.262064e+06,1434.515028,2544.620441,0.582922,NYSE,NaN,United States
3,1031123,2007,5.752388e+06,1392.225544,2267.907084,0.548892,NYSE,NaN,United States
4,1031123,2008,5.278202e+06,1607.810634,1639.743009,0.499488,NYSE,NaN,United States
...,...,...,...,...,...,...,...,...,...
1528,9991443,2020,1.164779e+05,547.711000,260.177000,1.223000,OB,NaN,Norway
1529,10421857,2016,4.615512e+04,120.874412,167.828007,0.439520,ATSE,True,Greece
1530,10421857,2017,5.176191e+04,146.548195,168.034624,0.475739,ATSE,True,Greece
1531,10421857,2018,7.929564e+03,23.813520,156.086722,0.468749,ATSE,True,Greece


In [239]:
df_panel_all[['ln_absolute_ghg_scope_1', 'ln_intensity_ghg_scope_1']] = np.log(df_panel_all[['absolute_ghg_scope_1', 'intensity_ghg_scope_1']])
df_panel_all

,entity_id,year,absolute_ghg_scope_1,absolute_ghg_scope_2,intensity_ghg_scope_1,intensity_ghg_scope_2,exchange,is_eu,country,ln_absolute_ghg_scope_1,ln_intensity_ghg_scope_1
0,1031123,2004,6.094485e+06,1582.414375,3167.529750,0.822439,NYSE,NaN,United States,15.622895,8.060707
1,1031123,2005,5.547608e+06,1891.407577,2503.925747,0.853691,NYSE,NaN,United States,15.528877,7.825615
2,1031123,2006,6.262064e+06,1434.515028,2544.620441,0.582922,NYSE,NaN,United States,15.650020,7.841737
3,1031123,2007,5.752388e+06,1392.225544,2267.907084,0.548892,NYSE,NaN,United States,15.565126,7.726613
4,1031123,2008,5.278202e+06,1607.810634,1639.743009,0.499488,NYSE,NaN,United States,15.479096,7.402295
...,...,...,...,...,...,...,...,...,...,...,...
1528,9991443,2020,1.164779e+05,547.711000,260.177000,1.223000,OB,NaN,Norway,11.665457,5.561362
1529,10421857,2016,4.615512e+04,120.874412,167.828007,0.439520,ATSE,True,Greece,10.739763,5.122940
1530,10421857,2017,5.176191e+04,146.548195,168.034624,0.475739,ATSE,True,Greece,10.854410,5.124170
1531,10421857,2018,7.929564e+03,23.813520,156.086722,0.468749,ATSE,True,Greece,8.978353,5.050412


In [242]:
unit_col_name='entity_id'
time_period_col_name='year'

y_var_name = 'ln_absolute_ghg_scope_1'
X_var_names = ['C(is_eu)']

regr_formula = y_var_name + ' ~ ' + ' + '.join(X_var_names) + ' + C(year) + C(country)'

panel_fe_model = smf.ols(formula=regr_formula, data=df_panel_all) # df_panel  df_panel_long_hist
panel_fe_model = panel_fe_model.fit()
print(panel_fe_model.summary())
stargazer_to_html(Stargazer([panel_fe_model]).render_html(), 'panel_fe_model_7.html')

                               OLS Regression Results                              
Dep. Variable:     ln_absolute_ghg_scope_1   R-squared:                       0.357
Model:                                 OLS   Adj. R-squared:                  0.295
Method:                      Least Squares   F-statistic:                     5.797
Date:                     Fri, 05 May 2023   Prob (F-statistic):           6.70e-16
Time:                             14:20:42   Log-Likelihood:                -678.36
No. Observations:                      322   AIC:                             1415.
Df Residuals:                          293   BIC:                             1524.
Df Model:                               28                                         
Covariance Type:                 nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

True

In [244]:
print(panel_fe_model.summary().tables[1])

                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        7.7060      0.262     29.419      0.000       7.190       8.222
C(is_eu)[T.True]                 7.7060      0.262     29.419      0.000       7.190       8.222
C(year)[T.2005]                 -0.2425      0.761     -0.318      0.750      -1.741       1.256
C(year)[T.2006]                 -0.2149      0.761     -0.282      0.778      -1.714       1.284
C(year)[T.2007]                 -0.1702      0.791     -0.215      0.830      -1.728       1.388
C(year)[T.2008]                 -0.1172      0.775     -0.151      0.880      -1.643       1.409
C(year)[T.2009]                 -0.4124      0.741     -0.556      0.578      -1.871       1.046
C(year)[T.2010]                 -0.3926      0.741     -0.530      0.597      -1.852       1.066
C(year)[T.2011]               